# Curso de Atualização em Health Data Science Analytics
## Novembro/2019

## Análise Exploratória de Dados


# Apresentação
## Prof. Me Fernando Sequeira Sousa



*   Bacharel em Informática Biomédica - FMRP/FFCLRP - USP
*   Mestre em Ciência - Área de Gestão e Informática em Saúde - DIS - UNIFESP
*   Lead Data Engineer - Optimum Soluções
*   Professor Faculdade Impacta de Tecnologia
   *   Graduação - ADS, SI, BD, GTI
   *   Pós-graduação - Big Data e Business Analytics
*   Principais Disciplinas: 
   *   Web Mining 
   *   Linguagem SQL e Linguagem NoSQL 
   *   Projeto de Big Data Analytics 
   *   Linguagem de Programação 
   *   Tecnologias Web 
   *   Desenvolvimento para Dispositivos Móveis 
   *   Oficina Projeto de Empresa 
   *   Trabalho de Conclusão de Curso
   
# Objetivos

O objetivo do curso é apresentar as principais técnicas estatísticas para descrição e
exploração de dados, utilizar os pacotes estatísticos com a linguagem python e promover habilidades de interpretação de resultados de análises descritivas e exploratórias.


# Conteúdo Programático
   
*   Tipos de variáveis e organização de banco de dados
*   Representação de variáveis qualitativas: 
   *   tabelas de frequências;
   *   gráficos de barras;
   *   gráficos de setores.
*   Medidas descritivas de dados quantitativos:
   *   posição;
   *   dispersão;
   *   assimetria;
   *   concentração.
* Representação de variáveis quantitativas:
   *   tabelas de frequências;
   *   histograma, gráfico de caixas;
   *   ramo-e-folhas;
   *   gráfico de pontos.
*   Tabelas de contingência e medidas de associação entre variáveis qualitativas;
*   Associação entre variáveis quantitativas e qualitativas;
*   Representação gráfica de dados multidimensionais;
*   Análise de agrupamentos. 
   

   





# Preparação do Ambiente
## Google colab
Acesse: https://colab.research.google.com e faça login com sua conta do Google
1.   Selecione a opção "GOOGLE DRIVE"
2.   Selecione a opção "NOVO PYTHON 3 NOTEBOOK"

![alt text](https://docs.google.com/uc?export=download&id=1WjrNw5VjwSxp7AzaNFcjHty_CR91Lyaq)


O notebook será aberto. No notebook faça o seguinte:
1.   Altere no nome do arquivo
2.   clique ao lado do "play", escreva o código abaixo e aperte o "play":


```
print('Olá mundo')
```

Veja o resultado logo abaixo.


![alt text](https://docs.google.com/uc?export=download&id=16uB-y_fcjs9dYr41YVh1vVn3ExBf_Xc5)


# Introdução

A análise exploratória de dados é uma forma de analisar um conjunto de dados para sumarizar suas características. Usualmente são utilizados métodos visuais para representar os dados, como tabelas e gráficos.


## Dados

![alt text](https://docs.google.com/uc?export=download&id=1wfLjY-ENixc2Bzyb1OmmwpnjhTZ-o5eR) 

## Informação

![alt text](https://docs.google.com/uc?export=download&id=1FiHlK-iIw4qT8bsyO0CMClv3zb6OCF_y) 
*   A média de idade das mãed com nascidos vivos em 2015 foi de 27,3 anos
*   Em 205, ouve mais nascidos vivos do sexo masculino  

## Representação
![alt text](https://docs.google.com/uc?export=download&id=1OZ6sY2ZvI9VZB2KUmsghQeIyBYRLZAuu) 

*   Histograma de idade das mães de nascidos vivos em 2015



# Tipos de Variáveis e Organização de Bases de Dados

<br/>

![alt text](https://docs.google.com/uc?export=download&id=1BMdfOUqg2zr0_unJcjS3LPIuSgb5HpsZ) 



## Exercício - Tipos de Variáveis

Classifique cada uma das varáveis da base de dados de Nascidos Vivos do Acre em 2017 nas categorias acima

# Variáveis Quantitativas e Medidas Descritivas

# Variáveis Quantitativas e Representações

# Variáveis Qualitativas e Representações

# Variáveis Qualitativas - Tabela de Contingência e Medidas de Associação

# Associação entre Variáveis Qualitativas e Quantitativas

# Análise de Agrupamentos